In [0]:
from config.paths import SECRETS_FILE
from src.utils.locate_filepaths import storage_filepaths
import json
from src.etl.climate.extract import extract_dmi_metObs
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import current_timestamp

dbutils.widgets.text("source", "")
source = dbutils.widgets.get("source")
print(f"Notebook triggered by: {source}")

# Loading storage filepaths
filepaths = storage_filepaths()

In [0]:
# Loading dmi api key
with open(SECRETS_FILE) as f:
    secrets = json.load(f)

In [0]:
# Schema for dmi-climate data
schema = StructType([
    StructField('created', StringType(), True),
    StructField('observed', StringType(), True),
    StructField('parameterId', StringType(), True),
    StructField('stationId', StringType(), True),
    StructField('value', DoubleType(), True)
])

df_climate_dmi = spark.createDataFrame([], schema)

In [0]:
# Fetching data from dmi
url = "https://dmigw.govcloud.dk/v2/metObs/collections/observation/items"
parameterIds = ["sun_last1h_glob","temp_mean_past1h","temp_soil_mean_past1h","precip_past1h"]
bbox="12.3,55.6,12.7,56.0"
period='latest-month'
limit=1000
secret_key=secrets["metObs-api-key"]

for i in parameterIds:
    data = extract_dmi_metObs(secret_key, url=url, parameterId=i, bbox=bbox, period=period, limit=limit)
    features = data.get('features')
    records = [item['properties'] for item in features]
    temp_df = spark.createDataFrame(records, schema)
    df_climate_dmi = df_climate_dmi.union(temp_df)

df_climate_dmi = df_climate_dmi.withColumn("ingestion_timestamp", current_timestamp())
df_climate_dmi.write.mode('overwrite').partitionBy("ingestion_timestamp").parquet(f"{filepaths['bronze']}/dmi_climate/daily_extract/")